## 1. Configuración Inicial

In [1]:
import sys
sys.path.insert(0, '..')

from discrete_logistics.core import Problem, Item, Bin, Solution
from discrete_logistics.core.instance_generator import InstanceGenerator
from discrete_logistics.algorithms.greedy import FirstFitDecreasing, BestFitDecreasing, RoundRobinGreedy
from discrete_logistics.algorithms.metaheuristics import SimulatedAnnealing, GeneticAlgorithm

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 2. Crear una Instancia del Problema

In [2]:
# Crear items manualmente
items = [
    Item(0, weight=10, value=25),
    Item(1, weight=15, value=35),
    Item(2, weight=8, value=20),
    Item(3, weight=12, value=28),
    Item(4, weight=6, value=15),
    Item(5, weight=18, value=40),
    Item(6, weight=9, value=22),
    Item(7, weight=14, value=32),
    Item(8, weight=7, value=18),
    Item(9, weight=11, value=26),
]

# Crear problema con capacidades individuales por bin
problem = Problem(
    items=items,
    num_bins=3,
    bin_capacities=[40.0, 45.0, 35.0],  # Cada bin tiene su propia capacidad
    name="tutorial_example"
)

print(f"Problema: {problem.name}")
print(f"Número de items: {len(problem.items)}")
print(f"Número de bins: {problem.num_bins}")
print(f"Capacidades por bin: {problem.bin_capacities}")
print(f"Peso total: {sum(item.weight for item in problem.items)}")
print(f"Valor total: {sum(item.value for item in problem.items)}")

Problema: tutorial_example
Número de items: 10
Número de bins: 3
Capacidades por bin: [40.0, 45.0, 35.0]
Peso total: 110
Valor total: 261


## 3. Generar Instancias Aleatorias

In [6]:
# Usar el generador de instancias
generator = InstanceGenerator(seed=42)

random_problem = generator.generate_uniform(
    n_items=20,
    num_bins=4,
    weight_range=(5, 30),
    value_range=(10, 50),
    capacity_factor=1.5,      # controla la capacidad base; ajusta según necesites
    capacity_variation=0.0,   # si quieres capacidades heterogéneas, pon >0
    name="random_uniform"
)

print(f"Instancia generada con {len(random_problem.items)} items")

Instancia generada con 20 items


## 4. Resolver con Algoritmos Greedy

In [7]:
# First Fit Decreasing
ffd = FirstFitDecreasing()
solution_ffd = ffd.solve(problem)

# Round Robin Greedy (reemplaza LPTBalanced)
rrg = RoundRobinGreedy()
solution_rrg = rrg.solve(problem)

def calculate_objective(solution):
    bin_values = [sum(item.value for item in bin_obj.items) for bin_obj in solution.bins]
    return max(bin_values) - min(bin_values)

print("Resultados Greedy:")
print(f"  FFD - Objetivo: {calculate_objective(solution_ffd):.2f}")
print(f"  Round Robin - Objetivo: {calculate_objective(solution_rrg):.2f}")

Resultados Greedy:
  FFD - Objetivo: 14.00
  Round Robin - Objetivo: 14.00


## 5. Resolver con Metaheurísticas

In [8]:
# Simulated Annealing
sa = SimulatedAnnealing(
    initial_temp=100,
    cooling_rate=0.995,
    max_iterations=5000
)
solution_sa = sa.solve(problem)

# Genetic Algorithm
ga = GeneticAlgorithm(
    population_size=50,
    generations=100,
    mutation_rate=0.1
)
solution_ga = ga.solve(problem)

print("Resultados Metaheurísticas:")
print(f"  SA - Objetivo: {calculate_objective(solution_sa):.2f}")
print(f"  GA - Objetivo: {calculate_objective(solution_ga):.2f}")

Resultados Metaheurísticas:
  SA - Objetivo: 2.00
  GA - Objetivo: 2.00


## 6. Visualización de Solución

In [9]:
def visualize_solution(solution, title="Solution"):
    """Crear visualización de una solución con capacidades individuales."""
    fig = make_subplots(
        rows=1, cols=len(solution.bins),
        subplot_titles=[f"Bin {i+1} (Cap: {b.capacity:.0f})" for i, b in enumerate(solution.bins)]
    )
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    
    for bin_idx, bin_obj in enumerate(solution.bins):
        y_offset = 0
        for item_idx, item in enumerate(bin_obj.items):
            fig.add_trace(
                go.Bar(
                    x=[f"Bin {bin_idx+1}"],
                    y=[item.weight],
                    base=y_offset,
                    name=f"Item {item.id}",
                    marker_color=colors[item.id % len(colors)],
                    text=f"Item {item.id}<br>w={item.weight}<br>v={item.value}",
                    textposition='inside',
                    showlegend=bin_idx == 0
                ),
                row=1, col=bin_idx+1
            )
            y_offset += item.weight
        
        # Añadir línea de capacidad
        fig.add_hline(y=bin_obj.capacity, line_dash="dash", line_color="red",
                      row=1, col=bin_idx+1, annotation_text=f"Cap: {bin_obj.capacity}")
    
    fig.update_layout(
        title=title,
        barmode='stack',
        height=400
    )
    
    return fig

# Visualizar mejor solución
best_solution = min(
    [solution_ffd, solution_rrg, solution_sa, solution_ga],
    key=calculate_objective
)
fig = visualize_solution(best_solution, "Mejor Solución Encontrada")
fig.show()

## 7. Comparación de Algoritmos

In [10]:
results = {
    'FFD': calculate_objective(solution_ffd),
    'RoundRobin': calculate_objective(solution_rrg),
    'SA': calculate_objective(solution_sa),
    'GA': calculate_objective(solution_ga)
}

fig = go.Figure(data=[
    go.Bar(
        x=list(results.keys()),
        y=list(results.values()),
        marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    )
])

fig.update_layout(
    title='Comparación de Algoritmos',
    xaxis_title='Algoritmo',
    yaxis_title='Valor Objetivo (menor es mejor)',
    height=400
)

fig.show()

## 8. Análisis de Distribución

In [11]:
def analyze_solution(solution, name):
    """Analizar la distribución de una solución."""
    bin_weights = [sum(item.weight for item in b.items) for b in solution.bins]
    bin_values = [sum(item.value for item in b.items) for b in solution.bins]
    bin_capacities = [b.capacity for b in solution.bins]
    
    print(f"\n{name}:")
    print(f"  Capacidades: {[f'{c:.1f}' for c in bin_capacities]}")
    print(f"  Pesos por bin: {[f'{w:.1f}' for w in bin_weights]}")
    print(f"  Valores por bin: {[f'{v:.1f}' for v in bin_values]}")
    print(f"  Utilización: {[f'{w/c*100:.1f}%' for w, c in zip(bin_weights, bin_capacities)]}")
    print(f"  Desviación std de valores: {np.std(bin_values):.2f}")
    print(f"  Objetivo (diferencia): {max(bin_values) - min(bin_values):.2f}")

analyze_solution(solution_ffd, "FFD")
analyze_solution(solution_rrg, "Round Robin Greedy")
analyze_solution(solution_sa, "Simulated Annealing")
analyze_solution(solution_ga, "Genetic Algorithm")


FFD:
  Capacidades: ['40.0', '45.0', '35.0']
  Pesos por bin: ['35.0', '40.0', '35.0']
  Valores por bin: ['83.0', '96.0', '82.0']
  Utilización: ['87.5%', '88.9%', '100.0%']
  Desviación std de valores: 6.38
  Objetivo (diferencia): 14.00

Round Robin Greedy:
  Capacidades: ['40.0', '45.0', '35.0']
  Pesos por bin: ['35.0', '40.0', '35.0']
  Valores por bin: ['83.0', '96.0', '82.0']
  Utilización: ['87.5%', '88.9%', '100.0%']
  Desviación std de valores: 6.38
  Objetivo (diferencia): 14.00

Simulated Annealing:
  Capacidades: ['40.0', '45.0', '35.0']
  Pesos por bin: ['37.0', '38.0', '35.0']
  Valores por bin: ['87.0', '88.0', '86.0']
  Utilización: ['92.5%', '84.4%', '100.0%']
  Desviación std de valores: 0.82
  Objetivo (diferencia): 2.00

Genetic Algorithm:
  Capacidades: ['40.0', '45.0', '35.0']
  Pesos por bin: ['37.0', '38.0', '35.0']
  Valores por bin: ['87.0', '88.0', '86.0']
  Utilización: ['92.5%', '84.4%', '100.0%']
  Desviación std de valores: 0.82
  Objetivo (diferencia)

## 9. Conclusiones

Este tutorial ha demostrado:

1. Cómo crear instancias del problema con **capacidades individuales por bin**
2. Cómo usar diferentes algoritmos para resolver el problema
3. Cómo visualizar y comparar soluciones considerando las capacidades heterogéneas
4. Cómo analizar la calidad y utilización de las soluciones

**Nota:** Cada bin puede tener una capacidad diferente, lo que añade flexibilidad al modelo para representar escenarios reales como vehículos con distintas capacidades de carga.

Para más información, consulte la documentación completa en el archivo `README.md`.